In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 978.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.9 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Tool Define

In [ ]:
from langchain.tools import tool
from typing import List
import requests

# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']

import requests

@tool
def search_spotify_playlists(query: str) -> dict:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns a JSON object with the top 5 playlists including name, description, image, link, owner, and track count.
    """
    client_id = "api_key_here"
    client_secret = "api_secret_here"

    try:
        token = get_spotify_token(client_id, client_secret)

        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return {"message": "No playlists found for your query."}

        result = {
            "query": query,
            "total": len(playlists),
            "playlists": []
        }

        for pl in playlists:
            if pl is None:
                continue

            result["playlists"].append({
                "name": pl.get('name'),
                "description": pl.get('description'),
                "external_url": pl.get('external_urls', {}).get('spotify'),
                "image": pl.get('images', [{}])[0].get('url'),
                "owner": {
                    "name": pl.get('owner', {}).get('display_name'),
                    "url": pl.get('owner', {}).get('external_urls', {}).get('spotify')
                },
                "tracks": {
                    "url": pl.get('tracks', {}).get('href'),
                    "total": pl.get('tracks', {}).get('total')
                }
            })

        return result

    except Exception as e:
        return {"error": str(e)}


 


# Tool Run

In [ ]:
search_spotify_playlists.run("happy")

{'query': 'happy', 'total': 5, 'playlists': []}

In [ ]:
search_spotify_playlists.run("meditation")

{'query': 'meditation',
 'total': 5,
 'playlists': [{'name': 'Tibetan Healing Music, Zen Meditation, Stress Relief, Relaxing Music, Tibetan Bowls, Indian Flute',
   'description': 'Tibetan Healing Sounds - Cleanse the aura and space. Indian Flute Meditation Music, Yoga Music, Positive Vibes, Peace Of Mind . Bamboo Flute Music, Positive Energy Vibration, Cleanse Negative Energy. ',
   'external_url': 'https://open.spotify.com/playlist/0mnWqdNjnmb0AqNmXzQ0Vz',
   'image': 'https://image-cdn-fa.spotifycdn.com/image/ab67706c0000d72cadec9d86e0f9f46e02341c83',
   'owner': {'name': 'Steven Jones',
    'url': 'https://open.spotify.com/user/5vkly7zasn8eqxxlnhmuivgq8'},
   'tracks': {'url': 'https://api.spotify.com/v1/playlists/0mnWqdNjnmb0AqNmXzQ0Vz/tracks',
    'total': 338}}]}

# Agent Definition

In [ ]:
 
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlists],
    prompt=(
        "You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists "
        "based on their search queries using the tool provided.\n\n"
        "🧠 Think step-by-step, and always call the `search_spotify_playlists` tool with the user query.\n"
        "📦 When returning results to the user, format the final output in **JSON** using this structure:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "total": integer,\n'
        '  "playlists": [\n'
        "    {\n"
        '      "name": string,\n'
        '      "description": string,\n'
        '      "external_url": string,\n'
        '      "image": string,\n'
        '      "owner": {"name": string, "url": string},\n'
        '      "tracks": {"url": string, "total": integer}\n'
        "    },\n"
        "    ...\n"
        "  ]\n"
        "}\n\n"
        "Only return JSON. Do not add any extra commentary or text outside of the JSON structure."
    ),
    name="spotify_assistant"
)


# Agent Invoke

# Happy

In [ ]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="happy")
            ]
    }
    )
response["messages"]

[HumanMessage(content='happy', additional_kwargs={}, response_metadata={}, id='66d36f85-8745-4dfb-9e31-24c67fdd3260'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cZSeE4dVor2d3UCsXeda1Zgd', 'function': {'arguments': '{"query":"happy"}', 'name': 'search_spotify_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 108, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BjIpdFhXFXwz3pDeKFBA09SgfP2gB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--043ef8b0-e48f-4a42-828d-8de0c03b96e6-0', tool_calls=[{'name': 'search_spotify_playlists', 'args': {'query': 'hap

In [ ]:
response["messages"][-1].content

'It looks like there are no specific playlists found for the keyword "happy." Would you like to try a different keyword or perhaps search for another mood or genre?'

# Meditation

In [ ]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="meditation")
            ]
    }
    )
response["messages"]

[HumanMessage(content='meditation', additional_kwargs={}, response_metadata={}, id='e77b23df-1ed8-4a0d-b456-850be7ecdc12'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zA0Wq43iAThS45ilaiqnAeqM', 'function': {'arguments': '{"query":"meditation"}', 'name': 'search_spotify_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 257, 'total_tokens': 275, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjIrtTAxVq9IB3Jqh37YvXAJgCgNe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--8baa2de9-531c-4f0c-b588-83ae08b691e3-0', tool_calls=[{'name': 'search_spotify_playlists', 'args': {'qu

In [ ]:
response["messages"][-1].content

'{\n  "query": "meditation",\n  "total": 1,\n  "playlists": [\n    {\n      "name": "Tibetan Healing Music, Zen Meditation, Stress Relief, Relaxing Music, Tibetan Bowls, Indian Flute",\n      "description": "Tibetan Healing Sounds - Cleanse the aura and space. Indian Flute Meditation Music, Yoga Music, Positive Vibes, Peace Of Mind . Bamboo Flute Music, Positive Energy Vibration, Cleanse Negative Energy.",\n      "external_url": "https://open.spotify.com/playlist/0mnWqdNjnmb0AqNmXzQ0Vz",\n      "image": "https://image-cdn-fa.spotifycdn.com/image/ab67706c0000d72cadec9d86e0f9f46e02341c83",\n      "owner": {"name": "Steven Jones", "url": "https://open.spotify.com/user/5vkly7zasn8eqxxlnhmuivgq8"},\n      "tracks": {"url": "https://api.spotify.com/v1/playlists/0mnWqdNjnmb0AqNmXzQ0Vz/tracks", "total": 338}\n    }\n  ]\n}'

# Sleep

In [ ]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="I can't sleep well.")
            ]
    }
    )
response["messages"]

[HumanMessage(content="I can't sleep well.", additional_kwargs={}, response_metadata={}, id='bd8024fe-d636-452d-9181-e4f04e279629'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SG0cZVsltVysh63Z1hZQ5TbR', 'function': {'arguments': '{"query":"sleep"}', 'name': 'search_spotify_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 260, 'total_tokens': 277, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjIs8DBT0ztlBfOQMuDh6YI69vywu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--7c658308-b53b-4a38-b51f-8c283c2ae5e9-0', tool_calls=[{'name': 'search_spotify_playlists', 'args': 

In [ ]:
response["messages"][-1].content

'{\n  "query": "sleep",\n  "total": 5,\n  "playlists": [\n    {\n      "name": "Fall into Sleep Instantly",\n      "description": "",\n      "external_url": "https://open.spotify.com/playlist/6L0keGopfWZMQ76K4O8bX1",\n      "image": "https://mosaic.scdn.co/640/ab67616d00001e0290554681311f9f37ea75e658ab67616d00001e029f83f202a39a490012d4c95bab67616d00001e02d2242b8f3ec2c01f10cde461ab67616d00001e02df44455933077ed472437b3d",\n      "owner": {\n        "name": "J",\n        "url": "https://open.spotify.com/user/22mkq2xd4hdinjthxp52qx7ui"\n      },\n      "tracks": {\n        "url": "https://api.spotify.com/v1/playlists/6L0keGopfWZMQ76K4O8bX1/tracks",\n        "total": 48\n      }\n    },\n    {\n      "name": "Thunderstorm Sounds - 10 Hours for Sleeping",\n      "description": "I created this Thunderstorm Sounds Playlist for Relaxing Sleep, Deep Sleep and ASMR Sleep. It\'s a variety of different compositions I created out of Rain and Thunder Sounds I recorded. I hope you enjoy it. Happy Rela

# Class Local

# Spotify Tool. PY

In [ ]:
import os
import requests
from langchain_core.tools import tool
from config.setting import CLIENT_ID, CLIENT_SECRET
from typing import List


# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']

@tool
def search_spotify_playlists(query: str) -> dict:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns a JSON object with the top 5 playlists including name, description, image, link, owner, and track count.
    """
    client_id = "api key"
    client_secret = "api secret"

    try:
        token = get_spotify_token(client_id, client_secret)

        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return {"message": "No playlists found for your query."}

        result = {
            "query": query,
            "total": len(playlists),
            "playlists": []
        }

        for pl in playlists:
            if pl is None:
                continue

            result["playlists"].append({
                "name": pl.get('name'),
                "description": pl.get('description'),
                "external_url": pl.get('external_urls', {}).get('spotify'),
                "image": pl.get('images', [{}])[0].get('url'),
                "owner": {
                    "name": pl.get('owner', {}).get('display_name'),
                    "url": pl.get('owner', {}).get('external_urls', {}).get('spotify')
                },
                "tracks": {
                    "url": pl.get('tracks', {}).get('href'),
                    "total": pl.get('tracks', {}).get('total')
                }
            })

        return result

    except Exception as e:
        return {"error": str(e)}





# Run spotify agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.spotify_agent import SpotifyAgent
def main():
    agent = SpotifyAgent()
    query = "I can't sleep well."
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Spotify Aget py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.spotify_tools import search_spotify_playlists
from config.setting import OPENAI_API_KEY

class SpotifyAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[search_spotify_playlists],
            prompt=(
                "You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists "
                "based on their search queries using the tool provided.\n\n"
                "🧠 Think step-by-step, and always call the `search_spotify_playlists` tool with the user query.\n"
                "📦 When returning results to the user, format the final output in **JSON** using this structure:\n\n"
                "{\n"
                '  "query": string,\n'
                '  "total": integer,\n'
                '  "playlists": [\n'
                "    {\n"
                '      "name": string,\n'
                '      "description": string,\n'
                '      "external_url": string,\n'
                '      "image": string,\n'
                '      "owner": {"name": string, "url": string},\n'
                '      "tracks": {"url": string, "total": integer}\n'
                "    },\n"
                "    ...\n"
                "  ]\n"
                "}\n\n"
                "Only return JSON. Do not add any extra commentary or text outside of the JSON structure."
            ),
            name="spotify_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content